<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/constraints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This python script is used to generate constraints from the WMT dataset (Turkish to English)

#### Installing and importing essential libraries

In [ ]:
!pip install datasets

In [ ]:
# importing necessary libraries

from datasets import load_dataset
from collections import defaultdict
from tqdm import tqdm

#### Loading the WMT dataset

In [ ]:
# loading the wmt dataset, turkish to english

ds = load_dataset("wmt/wmt16", "tr-en")

#### Helper functions to get the constraints

In [ ]:
def get_ngrams(src, n = 2, ):
  """
  The function returns the n_grams of length n, from the given sentence.

  args:
    src: takes in a sentence
    n: represents the size of ngram

  returns:
    returns the specified ngrams size"""

  src = src.split(" ")
  src = [tuple(src[i:i+n]) for i in range(len(src) - n + 1)]
  return src

In [ ]:
def constraints():

  """
  returns
    dict_pairs_count: total co-occurence of ngrams
    src_ngrams_count: dictionary containing occurences of unique ngrams in source language (turkish)
    tgt_ngrams_count: dictionary containing occurences of unique ngrams in target language (english)
    tgt_ngrams_count: total number of words in target language (english)
    tgt_ngrams_count: total number of words in source language (turkish)
  """

  src_ngrams_count = defaultdict(int)
  tgt_ngrams_count = defaultdict(int)
  dict_pairs = defaultdict(int)

  dict_pairs_count = 0
  total_src_words = 0
  total_tgt_words = 0


  count = 0
  n_gram = 2
  for doc in tqdm(ds["train"]["translation"]):

    # get the src, and tgt
    src = doc["tr"]
    tgt = doc["en"]

    # get the ngrams, default n = 2
    src_ngrams = get_ngrams(src, n = n_gram)
    tgt_ngrams = get_ngrams(tgt, n = n_gram)


    # count the occurences of each of the ngram
    for tgt_ngram in tgt_ngrams:
      total_tgt_words += n_gram
      tgt_ngrams_count[tgt_ngram] += 1

    for src_ngram in src_ngrams:
      total_src_words += n_gram
      src_ngrams_count[src_ngram] += 1

    # # count the combined co-occurence of the ngram
    for src_ngram in src_ngrams:
      for tgt_ngram in tgt_ngrams:
        dict_pairs_count+= min(src_ngrams_count[src_ngram], tgt_ngrams_count[tgt_ngram])

  return dict_pairs_count,  src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words



In [ ]:
def get_count_dict_pairs(filtered_src, filtered_tgt, src_ngrams_count, tgt_ngrams_count):
  """
  args:
    filtered_src: ngrams in source language, who have repeated more than 3 times
    filtered_tgt: ngrams in target language, who have repeated more than 3 times
    src_ngrams_count: dictionary containing occurences of unique ngrams in source language (turkish)
    tgt_ngrams_count: dictionary containing occurences of unique ngrams in target language (english)
  returns
    dict_pairs: returns the dictionary containing the ngrams and their count
  """
  dict_pairs = defaultdict(int)

  count = 0
  for doc in tqdm(ds["train"]["translation"]):

    # get the src, and tgt
    src = doc["tr"]
    tgt = doc["en"]

    # get the ngrams, default n = 2
    src_ngrams = get_ngrams(src)
    tgt_ngrams = get_ngrams(tgt)

    # # count the combined co-occurence of the ngram

    for src_ngram in src_ngrams:
      if src_ngram not in filtered_src: continue
      for tgt_ngram in tgt_ngrams:

        if tgt_ngram not in filtered_tgt: continue
        dict_pairs[(src_ngram, tgt_ngram)] += min(src_ngrams_count[src_ngram], tgt_ngrams_count[tgt_ngram])

  return dict_pairs
# dict_pairs =  get_count_dict_pairs()

In [ ]:
import math

def calculate_pmi(dict_pairs_count, dict_pairs, src_ngrams_count, tgt_ngrams_count, total_tgt_words, total_src_words):

  """
  This function returns the turkish sentences and corresponding english sentences of ngram size 2,
  it retains the ngrams whose score is greater than 0.9

  """
  ignored_count = 0
  pmi_dict = {}
  count = 0

  total_dict_pairs = dict_pairs_count

  for (src_ngram, tgt_ngram), count in tqdm(dict_pairs.items()):
    p_src_tgt = dict_pairs[(src_ngram, tgt_ngram)] / total_dict_pairs

    p_src = src_ngrams_count[src_ngram] / total_src_words
    p_tgt = tgt_ngrams_count[tgt_ngram] / total_tgt_words

    if p_src == 0 or p_tgt == 0:
      pmi = 0
    else:
      pmi = math.log2(p_src_tgt / (p_src * p_tgt))/ -math.log2(p_src_tgt)

    if pmi > 0.9:
      pmi_dict[(src_ngram, tgt_ngram)] = pmi
    else:
      ignored_count += 1

  return pmi_dict


In [ ]:
def get_constraints():

  """
  returns the constraints, who pmi value is greater than 0.9.
  """

  dict_pairs_count, src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words =  constraints()
  min_count = 3
  # dict_pairs_count, dict_pairs, src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words


  filtered_src = {k: v for k, v in src_ngrams_count.items() if v >= min_count}
  filtered_tgt = {k: v for k, v in tgt_ngrams_count.items() if v >= min_count}

  dict_pairs =  get_count_dict_pairs( src_ngrams_count=src_ngrams_count, tgt_ngrams_count= tgt_ngrams_count, filtered_src=filtered_src, filtered_tgt=filtered_tgt)



  pmi_scores = calculate_pmi(dict_pairs_count, dict_pairs, filtered_src, filtered_tgt, total_tgt_words, total_src_words)


  # dictionary to only return the pair with max PMI value; since, there might be a different tgt value for the same src.
  max_pmi_dict = {}


  for (src, tgt), pmi_value in pmi_scores.items():
      if src not in max_pmi_dict:
          max_pmi_dict[src] = (tgt, pmi_value)
      elif pmi_value > max_pmi_dict[src][1]:
        max_pmi_dict[src] = (tgt, pmi_value)

  return max_pmi_dict

#### call the get_constraints() function, to get the constraints

In [ ]:
pmi_scores = get_constraints()

In [ ]:
pmi_scores

In [ ]:
len(pmi_scores)

In [ ]:
max_entry = max(pmi_scores.items(), key=lambda x: x[1][1])
print("Entry with max value:", max_entry)